In [ ]:
import torch

# 1.Veri Seti Oluşturma

Text : emma , mia


In [ ]:
stoi = {'.': 0, 'a': 1, 'e': 2, 'i': 3, 'm': 4}

Context ve Target Oluşturma
- Kontekst boyutu 3 olsun (her bir giriş, 3 karakterlik bir kontekst içerir).
- Örnek veri için giriş (X) ve çıkış (Y) çiftlerini oluşturalım:
> emma için;
  - Kontekst: [., ., e] → Hedef: m → [0, 0, 2] → 4
  - Kontekst: [., e, m] → Hedef: m → [0, 2, 4] → 4
  - Kontekst: [e, m, m] → Hedef: a → [2, 4, 4] → 1
  - Kontekst: [m, m, a] → Hedef: . → [4, 4, 1] → 0

  > mia için;
  - Kontekst: [., ., m] → Hedef: i → [0, 0, 4] → 3
  - Kontekst: [., m, i] → Hedef: a → [0, 4, 3] → 1
  - Kontekst: [m, i, a] → Hedef: . → [4, 3, 1] → Ch0

In [ ]:
X = torch.tensor([
    [0,0,2],   # . . e ---> m
    [0,2,4],   # . e m ---> m
    [2,4,4],    # e m m --- > a
    [4,4,1],   # m m a --- > .
    [0,0,4],   #. . m ---- > i
    [0,4,3],  # . m i -----> a
    [4,3,1]    # m i a -----> .
])

Y = torch.tensor([4,4,1,0,3,1,0] )  # Hedef karakterlerin indexleri

Mini Batch Seçimi

- Mini-batch boyutu 2 olsun.Rastgele seçilen indexler [0,4]
- Mini batch girş ve çıkışları:

In [ ]:
Xb = X[[0,4]]
Yb = Y[[0,4]]
print(Xb,Yb,Xb.shape , Yb.shape)

tensor([[0, 0, 2],
        [0, 0, 4]]) tensor([4, 3]) torch.Size([2, 3]) torch.Size([2])


# 2.Kullanılacak PARAMETRELER

Model Parametreleri

- Modelin boyutlarını basitleştirelim
  1. Gömme vectör boyutu 2 ---> Her karaktere ait 2 boyut(özellik)
  2. Birinci katman nöron sayısı : 3 ----> n_hidden
  3. Karakter sayısı : 5 (. , a , e , i , m)---> vocab_size
  4. Context boyutu : 3 ----> İnput ----> Katmana girişi ---> Context boyutu x gömme vectör boyutu 3 * 2 = 6

In [ ]:
# Parametreler

# Gömme tablosu 5 karakter ve her biri 2 boyutlu
C = torch.tensor([
    [1.0,  0.5],   # . (0) ---> özellikler boyutlar
    [0.2 , 0.8],  # a (1)
    [0.4 , 0.6] ,  # e (2)
    [0.7 , 0.3] ,  # i (3)
    [0.9 , 0.1]   # m (4)
], dtype=torch.float32)


# ----------1.KATMAN ------------

# Birinci katman ( 6 giriş , 3 nöron)

W1 = torch.tensor([
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9],
    [1.0, 1.1, 1.2],
    [1.3, 1.4, 1.5],
    [1.6, 1.7, 1.8]
], dtype=torch.float32)

# Bias değerleri
b1 = torch.tensor([0.1,0.2,0.3] , dtype=torch.float32)

# ----------BATCHNORM--------------

bngain = torch.tensor([[1.0,1.0,1.0]] , dtype=torch.float32)
bnbias = torch.tensor([[0.0,0.0,0.0]] , dtype=torch.float32)


# -------------İKİNCİ KATMAN----------

# İkinci katman (3 nöron, 5 çıkış)
W2 = torch.tensor([
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.6, 0.5, 0.4, 0.3, 0.2],
    [0.7, 0.6, 0.5, 0.4, 0.3]
], dtype=torch.float32)

b2 = torch.tensor([0.1, 0.1, 0.1, 0.1, 0.1], dtype=torch.float32)

# 3.Forward Pass ( ileri Yayılım )

Şimdi mini-batch Xb = [[0,0,2],[0,0,4]] , Yb = [4,3] üzerinde ileri yayılımı adım adım hesaplayacağız

1. Adım : Gömme (Embedding)

- Her karakteri gömme vektörüne dönüştürelim

Xb[0] = [0 , 0 , 2] ----> ( .  .  e )

C[0] = [1.0 , 0.5] --->  .
C[0] = [1.0 , 0.5] --->  .
C[2] = [0.4 , 0.6] --->  e

Xb[1] = [0 , 0 , 4 ] ----> (. . m)

C[0] =[1.0, 5.0]  ---> .
C[0] =[1.0 , 0.5] ---> .
C[4] =[0.9 , 0.1] ---> m

In [ ]:
emb = C[Xb]   # (2 , 3 ,2) 2 örnek 3 context 2 boyut
#emb[0] = [[1.0,0.5],[1.0,0.5],[0.4,0.6]]
#emb[1]  [[1.0,0.5],[1.0,0.5],[0.9,0.1]]

In [ ]:
emb

tensor([[[1.0000, 0.5000],
         [1.0000, 0.5000],
         [0.4000, 0.6000]],

        [[1.0000, 0.5000],
         [1.0000, 0.5000],
         [0.9000, 0.1000]]])

In [ ]:
# BİRLEŞTİRME (FLATTEN)

embcat = emb.view(emb.shape[0],-1)  # (2,6) 2 örnek 2*3 = 6 boyut
# embcat[0] = [1.0,0.5,1.0,0.5,0.4,0.6]
# embcat[1] = [1.0,0.5,1.0,0.5,0.9,0.1]
embcat

tensor([[1.0000, 0.5000, 1.0000, 0.5000, 0.4000, 0.6000],
        [1.0000, 0.5000, 1.0000, 0.5000, 0.9000, 0.1000]])

# 4.Birinci Katman ( Linear )


hprebn = embcat @ W1 + b1

Matris Çarpımı : embcat (2,6)  @ (6,3) = (2,3)

Örnek Matris Çarpımı : embcat[0] = [1.0,0.5,1.0,0.5,0.4,0.6]

W1 = tensor([
        [0.1, 0.2, 0.3],
        [0.4, 0.5, 0.6],
        [0.7, 0.8, 0.9],
        [1.0, 1.1, 1.2],
        [1.3, 1.4, 1.5],
        [1.6, 1.7, 1.8]])

b1 = [0.1 , 0.2 , 0.3]

hprebn = [0,0] = 1.0*0.1+0.5*0.4+1.0*0.7+0.5*1.0+0.4*1.3+0.6*1.6 = 1.1+ 0.2 + 0.7 + 0.5 + 0.52 + 0.96 = 3.03
hprebn [0,1] = 1.0*0.2+0.5*0.5+1.0*0.8+0.5*1.1+0.4*1.4+0.6*1.7 = 0.2+0.25+0.8+ 0.55 + 0.56 + 1.02 = 3.38
hprebn[0,2] = 1.0*0.3 + 0.5*0.6 + 1.0*0.9 + 0.5*1.2 + 0.4*1.5+ 0.6*1.8 = 0.3 + 0.3 + 0.9 + 0.6 + 0.6 + 1.08 = 3.78

Bias Ekleme = hprebn[0] = [3.03 + 0.1 , 3.38 + 0.2 , 3.78 + 0.3] = [3.13 , 3.58  , 4.08]

embcat[1] = [1.0 , 0.5 , 1.0 , 0.5 , 0.9 , 0.1]

hprebn[1,0] = [1.0*0.1 + 0.5 * 0.4...] benzer şekilde hesaplanır
hprebn[1] = [3.08 , 3.48 , 3.88]



In [ ]:
hprebn = torch.tensor([
    [3.13,3.58,4.08],
    [3.08,3.48,3.88]
] , dtype= torch.float32)

# 5.BATCHNORM

--- Girişleri Normalize etmek

## 1.Ortalama Almak

In [ ]:
bnmeani = hprebn.mean(0 , keepdim = True)
# bnmeani[0] = (3.13 + 3.08) / 2  = 3.105
# bnmeani[1] = (3.58 + 3.48) / 2 = 3.53
# bnmeani[2] = (4.08 + 3.88) / 2 = 3.98
#bnmeani= tensor([[3.1050, 3.5300, 3.9800]])
bnmeani


tensor([[3.1050, 3.5300, 3.9800]])

## 2.Fark ve Kare

In [ ]:
bndiff = hprebn - bnmeani
# bndiff[0] = [3.13 - 3.105 , 3.58 - 3.53 , 4.08 - 3.98] = [0.025 , 0.05 , 0.1]
# bndiff[1] = [3.08 - 3.105 , 3.48 - 3.53 , 3.88 - 3.98] = [-0.025 , -0.05, -0.1]

bndiff2 = bndiff**2
# bndiff2[0] = [0.000625,0.0025,0.01]
# bndiff2[1] = [0.000625,0.0025,0.01]
bndiff2

tensor([[0.0006, 0.0025, 0.0100],
        [0.0006, 0.0025, 0.0100]])

## 3.Varyans (Bessel Düzeltmesi)

--- Varyans  : bndiff2 nin ortalaması : bessel düzeltmesi ortalama alırken eleman sayısının bir eksiğine bölünür


In [ ]:
bnvar = 1 / (2-1) * bndiff2.mean(0 , keepdim=True)  # Sütun toplanır eleman sayısının bir eksiğine böleriz

#bnvar[0] = 1/1 (0.000625 + 0.000625) = 0.00125
# bnvar[1] = 1/1 (0.0025 + 0.0025) = 0.005
# bnvar[2] = 1/1(0.01 + 0.01) = 0.02

bnvar = [[0.00125,0.005,0.02]]
bnvar_inv = (bnvar + 1e-5) ** -0.5
# bnvar_inv[0] = (0.00125 + 1e - 5) ^ (-0.5) ≈ 28.28
# bnvar_inv[1] = (0.005 + 1e-5) ^(-0.5) ≈ 14.14
#bnvar_inv[2] = (0.02 + 1e - 5) ^(0.5) ≈ 7.07
bnvar_inv = [[28.28,14.14,7.07]]

## 4.Normalizasyon

In [ ]:
bnraw = bndiff * bnvar_inv
# bnraw[0] = [0.025 * 28.28 , 0.05*14.14 , 0.1 * 7.07] ≈ [0.707 , 0.707 , 0.707 ]
#bnraw[1] = [-0.025 * 28.28 , -0.05 * 14.14 , -0.1 * 7.07] ≈ [-0.707 ,-0.707 , -0.707]

## 5.Ölçek ve Öteleme


In [ ]:
hpreact = bngain * bnraw + bnbias
# bngain = [[1,1,1,]] , bnbias = [0,0,0]
# hpreact = bnraw

hpreact = torch.tensor([
    [0.707,0.707,0.707],
    [-0.707,-0.707,-0.707]
],dtype = torch.float32)

# 6.Tanh Aktivasyonu

In [ ]:
h = torch.tanh(hpreact)
# tanh(0.707) ≈ 0.614 , tanh(-0.707) ≈ -0.614

h = torch.tensor([
    [0.614,0.614,0.614],
    [-0.614,-0.614,-0.614]
], dtype = torch.float32)

# 7.İkinci Katman(Linear)

logits = h @ W2 + b2
Matris Çarpımı = h(2,3) @ W2(3,5) = (2,5)

Örnek1 =>
 h = [[0.614,0.614,0.614],
    [-0.614,-0.614,-0.614]]

W2 = torch.tensor([
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.6, 0.5, 0.4, 0.3, 0.2],
    [0.7, 0.6, 0.5, 0.4, 0.3]
], dtype=torch.float32)

b2 = torch.tensor([0.1, 0.1, 0.1, 0.1, 0.1], dtype=torch.float32)

logits[0,0] = 0.614*0.5 + 0.614*0.6 + 0.614 * 0.7 + 0.1 = 0.307 + 0.3684 + 0.4298 + 0.1 = 1.2052
...
logits[0] = [1.2052 , 1.0828 , 0.9604 , 0.8380 , 0.7156]

Örnek 2 >
Aynı hesaplama [-0.614 * 0.5 + -0.614 * 0.6...

h[1] = [-1.2052 , -1.0828 , -0.9604 , -0.8380 , -0.7156]

In [ ]:
logits = torch.tensor([
    [1.2052, 1.0828, 0.9604, 0.8380, 0.7156],
    [-1.2052, -1.0828, -0.9604, -0.8380, -0.7156]
], dtype=torch.float32)

# 8.Kayıp Hesaplama(Cross - Entropy Loss)

## 1 . Sayısal kararlılık için Normalizasyon

In [ ]:
logit_maxes = logits.max(1 , keepdim = True).values
# logit_maxes[0] = 1.2052 , logit_maxes[1] = 0.7156

norm_logits = logits - logit_maxes

# norm_logits[0] = [0, -0.1224, -0.2448, -0.3672, -0.4896]
# norm_logits[1] = [-0.4896, -0.3672, -0.2448, -0.1224, 0]

## 2.Softmax

In [ ]:
counts = norm_logits.exp()
# counts[0] ≈ [1, 0.885, 0.783, 0.692, 0.613]
# counts[1] ≈ [0.613, 0.692, 0.783, 0.885, 1]

counts_sum = count.sum(1 , keepdim = True)
# counts_sum[0] ≈ 3.973, counts_sum[1] ≈ 3.973

count_sum_inv = counts_sum ** -1
# counts_sum_inv ≈ [0.252, 0.252]

probs = counts * counts_sum_inv
# probs[0] ≈ [0.252, 0.223, 0.197, 0.174, 0.154]
# probs[1] ≈ [0.154, 0.174, 0.197, 0.223, 0.252]

## 3.Log Olasılıklar

In [ ]:
logprobs =  probs.log()
# logprobs[0] ≈ [-1.378, -1.500, -1.623, -1.746, -1.869]
# logprobs[1] ≈ [-1.869, -1.746, -1.623, -1.500, -1.378]

## 4.Kayıp

- Yb = [4 ,3] (hedefler : m , i)
- Loss = - logprobs[range(2) , Yb].mean()
- logprobs[0,4] = -1.869 , logprobs[1,3] = -1.500
- loss = -(-1.869 + -1.500) / 2 = 1.6845

In [ ]:
loss = torch.tensor(1.6845)

# GERİYE DOĞRU MANUEL TÜREV İŞLEMLERİ

## KONTROL FONKSİYONU

In [ ]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

- dt = Manuel hesaplanan gradyan-türev-
- t.grad = Pytorch tarafından otomatik hesaplanan türev
- ex ---> exact = dt ile t.grad arasındaki tam eşleşme
- app---> approximate = dt ile t.grad arasındaki yaklaşık eşleşme
- maxdiff = dt ile t.grad arasındaki maximum fark

## Alıştırma Listesi

In [ ]:
# Exercisess

# cmp('logprobs', dlogprobs, logprobs)
# cmp('probs', dprobs, probs)
# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

Veri: Xb = [[0, 0, 2], [0, 0, 4]], Yb = [4, 3] (hedefler: m ve i).

İleri Yayılım Sonuçları (Özet):

In [ ]:
embcat = torch.tensor([[1.0, 0.5, 1.0, 0.5, 0.4, 0.6], [1.0, 0.5, 1.0, 0.5, 0.9, 0.1]], dtype=torch.float32)
hprebn = torch.tensor([[3.13, 3.58, 4.08], [3.08, 3.48, 3.88]], dtype=torch.float32)
bnmeani = torch.tensor([[3.105, 3.53, 3.98]], dtype=torch.float32)
bndiff = torch.tensor([[0.025, 0.05, 0.1], [-0.025, -0.05, -0.1]], dtype=torch.float32)
bndiff2 = torch.tensor([[0.000625, 0.0025, 0.01], [0.000625, 0.0025, 0.01]], dtype=torch.float32)
bnvar = torch.tensor([[0.00125, 0.005, 0.02]], dtype=torch.float32)
bnvar_inv = torch.tensor([[28.28, 14.14, 7.07]], dtype=torch.float32)
bnraw = torch.tensor([[0.707, 0.707, 0.707], [-0.707, -0.707, -0.707]], dtype=torch.float32)
hpreact = bnraw  # (bngain=1, bnbias=0)
h = torch.tanh(hpreact)  # [[0.614, 0.614, 0.614], [-0.614, -0.614, -0.614]]
logits = torch.tensor([[1.2052, 1.0828, 0.9604, 0.8380, 0.7156], [-1.2052, -1.0828, -0.9604, -0.8380, -0.7156]], dtype=torch.float32)
norm_logits = torch.tensor([[0.0000, -0.1224, -0.2448, -0.3672, -0.4896], [-0.4896, -0.3672, -0.2448, -0.1224, 0.0000]], dtype=torch.float32)
probs = torch.tensor([[0.252, 0.223, 0.197, 0.174, 0.154], [0.154, 0.174, 0.197, 0.223, 0.252]], dtype=torch.float32)
logprobs = torch.tensor([[-1.378, -1.500, -1.623, -1.746, -1.869], [-1.869, -1.746, -1.623, -1.500, -1.378]], dtype=torch.float32)
loss = torch.tensor(1.6845, dtype=torch.float32)

## 1.Alıştırma

- Loss 'un logprobs a göre türevi

- cmp('logprobs' , dlogprobs,logprobs)


- Ne Yapıyoruz ?

dloss / dlogprobs

loss = -logprobs[range(2) , Yb].mean()
loss = -(logprobs[0,4] , logprobs[1,3]) / 2

- Hesaplama

loss = -(logprobs[0,4] + logprobs[1,3])/2
     = -(-1.869 + -1.500) /2 = 1.6845

dloss / dlogprobs[0,4] = -0.5
dloss / dlogprobs[1,3] = -0.5

Niye ?
ax 'in x 'e göre türevi 'a' dır.
loss = -(logprobs[0,4] + logprobs[1,3]) / 2 --- > 1/2 katsayılarını dağıtırız.
     = -1/2(logprobs[0,4]) + -1/2(logprobs[1,3])

!!! n eleman sayısı için türevi -1/n !!!

In [ ]:
# Sonuç

dlogprobs = torch.zeros_like(logprobs)  # (2,5) 'lik sıfır matrisi oluşturuyoruz
dlogprobs[0,4] = -0.5
dlogprobs[1,3] = -0.5

#dlogprobs = [[0,0,0,0,-0.5],[0,0,0,-0.5,0]]

In [ ]:
# Kontrol
cmp('logprobs' , dlogprobs , logprobs)

## 2.Alıştırma
- cmp('probs', dprobs , probs)
- loss 'un probs a göre türevi

- Ne Yapıyoruz ?

logprobs = probs.log()

Yani

logprobs[i,j] = log(probs[i,j])

*Zincir Kuralı = dloss/dprobs = dloss/dlogprobs * dlogprobs/dprobs*

**Hesaplama**

**Logx 'in türevi 1/x**

O halde :

- dlogprobs[i,j] / dprobs[i,j] = 1 / probs[i,j]
- dloss / dprobs[i,j] = dlogprobs * 1 / probs[i,j]

Örnek-1)

- dlogprobs[0,4] = -0.5     , probs[0,4] = 0.154
- dprobs[0,4] = -0.5 * 1 / 0.154   ≈ -3.247
- dprobs[0,j] ---> Diğerleri için türev '0'

Örnek-2

- dlogprobs[1,3] = -0.5     , probs[1,3] = 0.223
- dprobs[1,3] = -0.5 * 1/ 0.223 ≈ -2.242
- dprobs[1,j] ---> Diğerleri için grad '0'




In [ ]:
# Sonuç

dprobs = dlogprobs / probs
# dprobs = [[0,0,0,0,-3.247],[0,0,0,-2.242,0]]

In [ ]:
# kontrol

cmp('probs',dprobs , probs)

##3.Alıştırma
- Loss ' un count_sum_inv ' e göre türevi
- cmp('count_sum_inv',dcount_sum_inv,count_sum_inv)

**Ne Yapıyoruz ?**

probs = counts * count_sum_inv

- Zincir Kuralı : (dloss / dprobs) * (dprobs / dcount_sum_inv)

**Hesaplama :**

- probs[i,j] = counts[i,j] * count_sum_inv[i]

Probs 'un count_sum_inv 'e göre türevi ax 'in x e göre türevini almaktır.Yani

dprobs[i,j] / dcount_sum_inv[j] = counts[i,j]

**Sonuç :** *dloss / dcount_sum_inv[i] = Σ_j(dprobs[i,j] * counts[i,j])*

**Örnek-1**

- counts[0] ≈ [1.000,0.885,0.783,0.692,0.613]
- dprobs[0] = [0,0,0,0,-3.247]
- dcount_sum_inv[0] = [0*1.000 + 0*0.885 + 0*0.783 + 0*0.692 + -3.247*0.613] ≈ -1.990

**Örnek-2**

- counts[1] ≈ [0.613,0.692,0.783,0.883,1.000]
- dprobs[1] = [0,0,0,-2.242,0]
- dcount_sum_inv[1] ≈ [0*0.613,0*0.692+0*0.783+(-2.242)*(0.883)+0*1.000] ≈ -1.984


In [ ]:
# sonuç

dcount_sum_inv = (dprobs * counts).sum(1 , keepdim = True)
# dcount_sum_inv = [[-1.990],[-1.984]]

In [ ]:
# kontrol
cmp('count_sum_inv' , dcount_sum_inv,count_sum_inv)

##4.Alıştırma
- loss 'um counts_sum 'a göre türevi
- cmp('counts_sum',dcounts_sum,counts_sum)

**Ne Yapıyoruz?**
counts_sum_inv = counts_sum ** -1

- x**-1 'in türevi -1x**-2 dir.

**Zincir Kuralı:** (dloss/dcount_sum_inv) / (dcount_sum_inv/dcounts_sum)

**Hesaplama:**

- *dcount_sum_inv / dcounts_sum = -1 /(counts_sum)**-2*
- *dloss / dcounts_sum[i] = dcounts_sum_inv[i] * -1/counts_sum[i]^2*
- counts_sum ≈ [3.973,3.973]

**Örnek-1:**

- dcount_sum_inv[0] = -1.990    , counts_sum[0] = 3.973
- dcounts_sum[0] = -1.990 * (-1/(3.973^2)) ≈ -1.990 *(-1/15.785) ≈ 0.126

**Örnek-2**
- dcount_sum_inv[1] = -1.984      , counts_sum[1] = 3.973
- dcounts_sum[1] = -1.984 * (-1/(3.973)^2) ≈ -1.984 * (-1/15.785) ≈ 0.126


In [ ]:
# sonuç
dcounts_sum = dcount_sum_inv * (-1/counts_sum**2)
# dcounts_sum = [[0.126] , [0.126]]

In [ ]:
# kontrol
cmp('counts_sum',dcounts_sum , counts_sum)

## 5.Alıştırma
- counts 'a göre türev
- cmp('counts' , dcounts , counts)

**Ne Yapıyoruz?**
- probs = counts * counts_sum_inv
- counts_sum = counts.sum(1)
- Yani counts'a iki yoldan gradyan gelir

1. probs üzerinden : dloss/dcounts = dprobs * dcounts_sum_inv
2. counts_sum üzerinden : counts_sum'un gradyanı counts'un her elemanına dağılır.

**Hesaplama**

-Yol 1:  dprobs[i,j] / dcounts[i,j] = counts_sum_inv[i]


- dcounts[i,j] += dprobs[i,j] * counts_sum_inv[i]
- counts_sum_inv ≈ [0.252,0.252]
- dcounts[0] += [0,0,0,0,-3.247] * 0.252 ≈ [0,0,0,0,-0.818]
- dcounts[1] += [0,0,0,-2.242,0] * 0.252 ≈ [0,0,0,-0.565,0]

-Yol 2: counts_sum[i] = Σ_j counts[i, j]

- dcounts_sum[i] / dcounts[i] = 1
- dcounts[i] += dcounts_sum[i]
- dcounts_sum ≈ [0.126 , 0.126]
- Her eleman için dcounts[0,j] += 0.126 , dcounts[1,j] += 0.126





In [ ]:
# sonuç

dcounts = dprobs * dcounts_sum_inv + dcounts_sum
# dcounts[0] ≈ [0.126, 0.126, 0.126, 0.126, -0.818 + 0.126] ≈ [0.126, 0.126, 0.126, 0.126, -0.692]
# dcounts[1] ≈ [0.126, 0.126, 0.126, -0.565 + 0.126, 0.126] ≈ [0.126, 0.126, 0.126, -0.439, 0.126]

In [ ]:
# kontrol
cmp('counts',dcounts,counts)

## 6.Alıştırma

- norm_logits'e göre türev
- cmp('norm_logits',dnorm_logits,norm_logits)

**Ne Yapıyoruz?**

- counts = norm_logits.exp()

**Zincir Kuralı:** (dloss/dcounts) * (dcounts / dnorm_logits)

**Hesaplama**

- dcounts[i,j] / dnorm_logits[i,j] = exp(norm_logits[i,j]) = counts[i,j]

- dloss / dnorm_logits[i,j] = dcounts[i,j] * counts[i,j]

- counts[0] ≈ [1.000,0.885,0.783,0.692,0.613]
- dcounts[0] ≈[0.216,0.216,0.216,0.216,-0.692]

- dnorm_logits[0] = [0.216*1.000,0.216*0.885,0.216*0.783,0.216*0.692,-0.692*0.613] ≈ [0.216,0.112,0.099,0.087,-0.424]

counts[1] ≈ [0.613,0.692,0.783,0.885,1.000]
dcounts[1] = [0.216,0.216,0.216,-0.439,0.216]
dnorm_logits[1] = [0.216*0.613,0.216*0.692,0.216*0.783,-0.439*0.885,0.216*1.000] = [0.077,0.087,0.099,-0.389,0.216]


In [ ]:
# sonuç
dnorm_logits = dcounts * counts
# dnorm_logits ≈ [[0.126, 0.112, 0.099, 0.087, -0.424], [0.077, 0.087, 0.099, -0.389, 0.126]]

In [ ]:
#kontrol
cmp('norm_logits', dnorm_logits, norm_logits)

## 7.Alıştırma

- logit_maxes 'e göre türev
- cmp('logit_maxes',dlogit_maxes,logit_maxes)

**Ne Yapıyoruz?**

- norm_logits = logits - logit_maxes

!!! logit_maxes[i] sadece norm_logits[i,j] yi etkiler

**Hesaplama**

- dnorm_logits[i,j] / dlogit_maxes[i] = -1

- dloss / dlogit_maxes[i] = Σ_j (dnorm_logits[i, j] * (-1))

- dnorm_logits[0] = [0.126,0.112,0.099,0.087,-0.424]

- dlogit_maxes[0] = -(0.126 + 0.112+0.099+0.087-0.424) ≈ -0.000

- dnorm_logits[1] = [0.077,0.087,0.099,-0.389,0.126]

-dlogit_maxes[1] = -(0.077 + 0.087+ 0.099 - 0.389 + 0.126) ≈ -0.000

In [ ]:
# sonuç

dlogit_maxes = -dnorm_logits.sum(1 , keepdim = True)
# dlogit_maxes ≈ [[0], [0]] (sayısal yuvarlama nedeniyle sıfır)

In [ ]:
# kkontrol
cmp('logit_maxes', dlogit_maxes, logit_maxes)

Not : logit_maxes 'in gradyanı 0(sıfır) çıkar , çünkü softmax sayısal kararlılık için kaydırma yapar  ve toplam gradyan sıfırlanır

## 8.Alıştırma

- logits' e göre türev
- cmp('logits',dlogits,logits)

**Ne yapıyoruz ?**

- norm_logits = logits - logit_maxes
- dnorm_logits[i,j] / dlogits[i,j] = 1

**Zincir Kuralı:** (dloss/dnorm_logits) * (dnorm_logits/dlogits)

**Hesaplama**

dloss / dlogits[i,j] = dnorm_logits[i,j] + dlogits_maxes[i] * dnorm_logits[i,j] / dlogits[i,j] = dlogit_maxes = 0 --- > Bu yüzden dlogits = dnorm_logits



In [ ]:
# sonuç
dlogits = dnorm_logits.clone()

In [ ]:
#kontrol
cmp('logits',dlogits,logits)

## 9.Alıştırma

- h'ye göre türev
- cmp('h',dh,h)

**Ne Napıyoruz?**

- logits = h @ W2 + b2
- dlogits[i,j] / dh[i,j] = W2[k,j]

**Hesaplama**

- dloss / dh[i,j] = Σ_j (dlogits[i,j] * W2[k,j] )

- dlogits[0] = [0.126,0.112,0.099,0.087,-0.424]
- dlogits[1]= [0,077,0.087,0.099,-0.389,0.126]

- W2 =[
      [0.5,0.4,0.3,0.2,0.1],
      [0.6,0.5,0.4,0.3,0.2],
      [0.7,0.6,0.5,0.4,0.3]] ----> 3x5 'lik matris
Transpose işlemi
- W2_T = [
  [0.5,0.6,0.7],
  [0.4,0.5,0.6],
  [0.3,0.4,0.5],
  [0.2,0.3,0.4],
  [0.1,0.2,0.3]
  ]-----> 5x3 'lik matris

dh = dnorm_logits @ W2_T
- dh[0,0] = [0.126*0.5 + 0.112*0.4 + 0.099*0.3 + 0.087*0.2 + (-.0.424)*0.1] ≈  [0.063 + 0.045 + 0.030 + 0.017 - 0.042] ≈ 0.113

- dh[0,1] = [0.126*0.6 + 0.112 * 0.5 + 0.099 * 0.4 + 0.087 * 0.3 + (-0.424)* 0.2] ≈ 0.113

- dh[0,2] = [0.126 * 0.7 + 0.112 * 0.6 + 0.099 * 0.5 + 0.087 * 0.4 + (-0.424)*0.3] ≈ [0.0882 + 0.0672 + 0.0495 + 0.0348 - 0,1272] ≈  0.114

- dh[1,0] = [0,077*0.5,0.087*0.4,0.099*0.3,-0.389*0.2,0.126*0.1] = [0.039 +0.035 + 0.030 -0.078 + 0.013 ] ≈ 0.039

In [ ]:
# Sonuç

dh = dnorm_logits @ W2 .T()
#dh ≈ [[0.113 , 0.113 , 0.152],[0.058,0.072,0.085]]

In [ ]:
# kontrol
cmp('h',dh,h)

## 10.Alıştırma
- W2 'ye göre türev
- cmp('W2',dW2 , W2)

**Ne Yapıyoruz?**

logits = h @ W2 + b2

**Zincir Kuralı :**  *dloss/ dW2[k,j] = (dloss / dlogits[i,j]) * dlogits[i,j] / dW2[k,j]*

- dlogits[i,j] / dW2[k,j] = h[i,k] --- > h.T()

- dloss / dW2[k,j] = Σ_i(dlogits[i,j]* h[i,k])

- dW2 = h.T() @ dlogits

**Hesaplama**

h = [
  [0.614 , 0.614 , 0.614],
  [-0.614 , -0.614 , -0.614]
] ---> (2x3) 'lük matris

h_T = [
  [0.614,-0.614],
  [0.614, -0.614],
  [0.614 , -0.614]
] ---> (3x2) 'lik matris

dlogits = [
  [0.126,  0.112 , 0.099 , 0.087,-0.424],
  [0.077 , 0.087 , 0.099, -0.389, 0.126]
] ---> (2x5) 'lik matris

**Neden Transpose**
- W2 nin boyutu (3,5) Yani dW2 'de (3,5) olmalı
- h(2,3) ve dlogits (2,5) çarpıldığında ;
- h.T (3,2) ile dlogits(2,5) çarpıldığında (3,5) lik bir matris üretir.

**Örnek-1**

- dW2[0,0] = [0.614 * 0.126 + -0.614 * 0.077] = [0.077 - 0.047 ] = [0.030]

- dW2[0,1] = [0.614*0.112 + -0.614 * 0.087] = [0.069 - 0.053 ] = [0.016]


In [ ]:
# Sonuç
dW2 = h.t() @ dlogits
# dW2 ≈ [[0.030, 0.028, 0.026, -0.077, 0.029],
#        [0.030, 0.028, 0.026, -0.077, 0.029],
#        [0.030, 0.028, 0.026, -0.077, 0.029]]


In [ ]:
# kontrol

cmp('W2' ,dW2 ,W2)

## 11.Alıştırma

- b2 'ye göre türev
- cmp('b2',db2,b2)

**Ne Yapıyoruz?**

- logits = h @ W2 + b2
- b2 (5,) boyutunda bir vektör

- dloss / db2[j] = Σ_i(dloss/dlogits[i,j])

**Hesaplama**

- Zincir Kuralı : dloss/dlogits * dlogits/db2

- dlogits[i,j] / db2[j] = 1
- dloss / db2[j] = Σ_i dlogits[i, j]

**Örnek:**

- dlogits = [[0.126, 0.112, 0.099, 0.087, -0.424], [0.077, 0.087, 0.099, -0.389, 0.126]]

db2[0] = dlogits[0,0] + dlogits[1,0] = 0.126 + 0.077 = 0.203
db2[1] = dlogits[0,1] + [1,1] = 0.112 + 0.087 = 0.199 ...

In [ ]:
# Sonuç

db2 = dlogits.sum(0)
# db2 ≈ [0.203, 0.199, 0.198, -0.302, -0.298]

In [ ]:
# kontrol
cmp('b2',db2 , b2)

## 12.Alıştırma
- hpreact ' e göre türev

**Ne yapıyoruz ?**

h = torch.tanh(hpreact)

dloss/dhpreact[i,j] = dloss/dh[i,j] * dh[i,j] / dhpreact[i,j]

**Hesaplama**

dh[i,j] / dhpreact[i,j] = 1 - tanh(hpreact[i,j]^2) = 1 - h[i,j]^2

dloss/ dhpreact[i,j] = dh[i,j] * (1 - h[i,j]^2)

**Örnek-1**

- h = [
  [0.614,0.614,0.614],
  [-0.614,-0.614,-0.614]
]

- 1 - h^2 = [1- 0.614^2 ,...] = [0.623,0.623,0.623]

- dh = [
  [0.113,0.133,0.152],
  [0.058,0.072,0.085]
]

- dpreact[0,0] = 0.113 * 0.623 = 0.070
- dpreact[0,1] = 0.133 * 0.623 = 0.083
.
.
.




In [ ]:
# Sonuç
dhpreact = dh * (1-h**2)

# dhpreact ≈ [[0.070, 0.083, 0.095], [0.036, 0.045, 0.053]]

In [ ]:
# Kontrol
cmp('hpreact', dhpreact,hpreact)

## 13.Alıştırma

- bngain 'e göre türev

** Ne Yapıyoruz **

- hpreact = bngain * bnraw + bnbias

- bngain (1,3) boyutunda

*dloss/ dbngain = dloss/dhpreact * dhpreact * dbngain*

**Hesaplama**

- dhpreact[i,j] / dbngain[0,j] = bnraw[i,j]

- dloss / dbngain[0,j] = Σ_i(dhpreact[i,j] * bnraw[i,j])

**Örnek-1**

- bnraw = [
  [0.707,0.707,0.707],
  [-0.707,-0.707,-0.707]
]

- dhpreact = [
  [0.070,0.083,0.095],
  [0.036,0.045,0.053]
]

dbngain[0,0] = 0.070 * 0.707 + 0.036*-0.707 = 0.050 - 0.025 = 0.025
...


In [ ]:
# Sonuç

dbngain = (dhpreact * bngain).sum(0, keepdim = True)

In [ ]:
# kontrol
 cmp('bngain' , dbngain,bngain)

## 14.Alıştırma

- bnbias ' a göre türev

**Ne Yapıyoruz?**

- dhpreact[i,j] / dbnbias[0,j] = 1

- dloss / dbnbias[0,j] = Σ_i dhpreact[i, j]

** Hesaplama **

dbnbias[0,0] = dhpreact[0,0] + dpreact[1,0] = 0.070 + 0.036 = 0.106

In [ ]:
# Sonuç

dbnbias = dpreact.sum(0, keepdim = True)
# dbnbias ≈ [[0.106, 0.128, 0.148]]


## 15.Alıştırma

- bnraw 'a göre türev

**Ne Yapıyoruz?**

- hpreact = bngain * bnraw + bnbias

- dhpreact[i,j] / dbnraw[i,j] = bngain[0,j]

** Hesaplama **

- bngain = [[1,1,1]]

Yani : dloss / draw[i,j] = dhpreact[i,j] * bngain[0,j] = dhpreact[i,j]

In [ ]:
# Sonuç

dbnraw = dhpreact * bngain
# dbnraw ≈ [[0.070, 0.083, 0.095], [0.036, 0.045, 0.053]]

In [ ]:
# kontrol
cmp('bnraw',dbnraw , bnraw)

## 16.Alıştırma

- bnvar_inv ' e göre türev

**Ne Yapıyoruz?**
- bnraw= bndiff * bnvar_inv
- dbnraw[i,j] / dbnvar_inv[0,j] = bndiff[i,j]

**Hesaplama**

- dloss / dbnvar_inv = Σ_i(dbnraw[i,j] * bndiff[i,j])

- bndiff = [
  [0.025,0.05,0.1],
  [-0.025 ,-0.05,-0.1]
]

- dbnraw = [
  [0.070,0.083,0.095],
  [0.036,0.045,0.053]
]

dbnvar_inv[0,0] = 0.070 * 0.025 + 0.036 *(-0.025) = 0.00175 - 0.0009 = 0.00085
...

In [ ]:
# Sonuç

dbnvar_inv = (bnraw * bndiff).sum(0,keepdim = True)
# dbnvar_inv ≈ [[0.00085, 0.00165, 0.00320]]

In [ ]:
# kontrol
cmp('bnvar_inv',dbnvar_inv,bnvar_inv)

## 17.Alıştırma

- bnvar ' a göre türev

**Ne Yapıyoruz?**

- bnvar_inv = (bnvar + 1e-5) ** -0.5

- dbnvar_inv / dbnvar = -0.5 * (bnvar + 1e-5)**-1.5

**Hesaplama**

**Örnek-1**
- bnvar = [
  [0.00125,0.005,0.02]
]

- dbnvar_inv[0,0] / dbnvar[0,0] = -0.5*(0.00125 + 1e-5)^(-1.5) =  -0.5 *(0.00126)^(-1.5) = -314.27

- dbnvar[0,0] = dbnvar_inv[0,0] * (-0.5 * (bnvar[0,0] + 1e-5)^(-1.5)) = 0.00085*(-314.27) = -0.267

In [ ]:
# sonuç
dbnvar = dbnvar_inv * (-0.5*(bnvar + 1e-5)**-1.5)
# dbnvar ≈ [[-0.267, -0.165, -0.080]]

In [ ]:
# kontrol
cmp('bnvar',dbnvar,bnvar)

## 18.Alıştırma

- bndiff2 'ye göre türev

** Ne yapıyoruz?**

- bnvar = (1/(n-1)) * bndiff2.sum(0,keepdim = True)

- dbnvar[0,j] / dbndiff2[i,j] = 1/(n-1) = 1(2-1) = 1

** Hesaplama **

- dloss / dbndiff2[i,j] = dbnvar[0,j] * 1
- dbndiff2[i,j] = dbnvar[0,j]


In [ ]:
# Sonuç

dbndiff2 = torch.zeros_like(bndiff2)
dbndiff2[:,:] = dbnvar
# dbndiff2 ≈ [[-0.267, -0.165, -0.080], [-0.267, -0.165, -0.080]]

In [ ]:
# kontrol
cmp('bndiff2',dbndiff2,bndiff2)

## 19.Alıştırma

- bndiff 'e göre türev

** Ne yapıyoruz?**

-1.Yol bndiff2 = bndiff ** 2
- 2.Yol bnraw = bndiff * bnvar_inv

İki yoldan türev gelir

1. bndiff2 üzerinden ; dbndiff2[i,j] / dbndiff[i,j] = 2.bndiff[i,j]


2. bnraw üzerinden ; dbnraw[i,j] / dbndiff[i,j] = bnvar_inv[0,j]

**Hesaplama**

- dloss / dbndiff[i,j] = dbndiff2[i,j] * (2*bndiff[i,j]) + dbnraw[i,j] * dbvar_inv[0,j]

**Örnek-1**

- bndiff = [[0.025,0.05,0.1],[-0.025,-0.05,-0.1]]

- bnvar_inv = [[28.28,14.14,7.07]]

- dbndiff[0,0] = (-0.267*2*0.025) + (0.070*28.28)= -0.01335 + 1.9796 = 1.966



In [ ]:
# Sonuç

dbndiff = dbndiff2 * (2*bndiff) + dbnraw * bnvar_inv
# dbndiff ≈ [[1.966, 1.174, 0.570], [-1.979, -1.186, -0.576]]

In [ ]:
# kontrol

cmp('bndiff',dbndiff,bndiff)

##20.Alıştırma

- bnmean 'a göre türev

**Ne yapıyoruz?**

- bndiff = hprebn - bnmeani

- dbndiff[i,j] / dbnmeani[0,j] = -1

**Hesaplama**

- dloss/ dbnmeani[0,j] = Σ_i(dbndiff[i,j] * -1)

** Örnek-1**

dbnmeani[0,0] = -(1.966 +(-1.979)) = -0.013

In [ ]:
# sonuç

dbnmeani = -dbndiff.sum(0 , keepdim = True)
# dbnmeani ≈ [[-0.013, -0.012, -0.006]]

In [ ]:
# kontrol

cmp('bnmeani', dbnmeani , bnmeani)

## 21.Alıştırma

- hprebn ' e göre türev

**Ne yapıyoruz?**

- bndiff = hprebn - bnmeani

- dbndiff[i,j] / dhprebn[i,j] = 1

- bnmeani = hprebn.mean(0,keepdim=True)

**Hesaplama**

- dloss / dhprebn[i,j] = dbndiff[i,j] + dbnmeani[0,j] * (1/n)

- n = 2 yani 1/n = 0.5

- dhprebn[0,0] = 1.966 + (-0.013*0.5) = 1.966 - 0.0065 = 1.9595

In [ ]:
# sonuç

dhprebn = dbndiff + dbnmeani * (1/2)

# dhprebn ≈ [[1.9595, 1.168, 0.567], [-1.9855, -1.192, -0.579]]

## 22.Alıştırma

- embcat 'e göre türev

**Ne Yapıyoruz?**

- hprebn = embcat @ W1 + b1

- dhprebn[i,j] / dembcat[i,k] = W1[k,j]

**Hesaplama**

- dloss/ dembcat[i,j] =Σ_j(dhprebn[i,j]* W1[k,j])

- W1 = [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], ..., [1.6, 1.7, 1.8]]
- dembcat = dhprebn @ W1.t()

In [ ]:
# sonuç

dembcat = dhprebn @ W1.t()


In [ ]:
# kontrol
cmp('embcat', dembcat,embcat)

## 23.Alıştırma

- W1 'e göre türev



**Ne Yapıyoruz?**

- dhprebn[i,j] / dW1[k,j] = embcat[i,k]

**Hesaplama**

- dloss / dW1[k,j] = Σ_i(dprebn[i,j] * embcat[i,k])

- dW1 = embcat.t() @ dhprebn

In [ ]:
# Sonuç
dW1 = embcat.t() @ dhprebn

In [ ]:
# kontrol
cmp('W1',dW1 , W1)

## 24.Alıştırma

- b1 'e göre türev

** Ne Yapıyoruz?**

hprebn = embcat @ W1 + b1

** Hesaplama **

dhprebn[i,j] / db1[j] =1

db1[j] = Σ_i dhprebn[i, j]

In [ ]:
#Sonuç

db1 = dhprebn.sum(0)

In [ ]:
# kontrol
cmp('b1',db1,b1)

## 25.Alıştırma

- emb'e göre türev


**Ne Yapıyoruz?**

- embcat = emb.view(emb.shape[0], -1)

- emb (2,3,2) , embcat(2,6)

**Hesaplama**

- dembcat 'i emb 'in şekline geri dönüştürürüz ; demb = dembcat.view(2,3,2)

In [ ]:
# Sonuç

demb = dembcat.view(2,3,2)

In [ ]:
# kontrol
cmp('emb',demb,emb)

## 26.Alıştırma

- C ' ye göre türev

**Ne Yapıyoruz?**

- emb = C[Xb]

- C(5,2) , Xb(2,3)

- demb 'i C ye geri yansıtırız

- Xb = [[0,0,2],[0,0,4]]
- C = C[0] , C[2] , C[4] 'e gradyan eklenir

- dC[0] += demb[0,0] + demb[0,1] + demb[1,1]

- dC[2] += demb[0,2]

- dC[4] += demb[1,2]

In [ ]:
# sonuç
dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    dC[Xb[i, j]] += demb[i, j]

In [ ]:
# kontrol
cmp('C', dC, C)